# Normalization and Tuning Neural Networks - Lab

## Introduction

For this lab on initialization and optimization, let's look at a slightly different type of neural network. This time, we will not perform a classification task as we've done before (Santa vs not santa, bank complaint types), but we'll look at a linear regression problem.

We can just as well use deep learning networks for linear regression as for a classification problem. Do note that getting regression to work with neural networks is a hard problem because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$, and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case!

## Objectives
You will be able to:
* Build a nueral network using keras
* Normalize your data to assist algorithm convergence
* Implement and observe the impact of various initialization techniques

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from keras import optimizers
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## Loading the data

The data we'll be working with is data related to facebook posts published during the year of 2014 on the Facebook's page of a renowned cosmetics brand.  It includes 7 features known prior to post publication, and 12 features for evaluating the post impact. What we want to do is make a predictor for the number of "likes" for a post, taking into account the 7 features prior to posting.

First, let's import the data set and delete any rows with missing data. Afterwards, briefly preview the data.

In [2]:
!ls

CONTRIBUTING.md       Facebook_metrics.txt  LICENSE.md
dataset_Facebook.csv  index.ipynb	    README.md


In [18]:
!head dataset_Facebook.csv

81370;Photo;2;1;4;4;;4188;7292;564;524;743;3861;2200;316;0;91;28;119

In [28]:
#Your code here; load the dataset and drop rows with missing values. Then preview the data.
df = pd.read_csv('dataset_Facebook.csv', delimiter=';')

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 498
Data columns (total 19 columns):
Page total likes                                                       495 non-null int64
Type                                                                   495 non-null object
Category                                                               495 non-null int64
Post Month                                                             495 non-null int64
Post Weekday                                                           495 non-null int64
Post Hour                                                              495 non-null int64
Paid                                                                   495 non-null float64
Lifetime Post Total Reach                                              495 non-null int64
Lifetime Post Total Impressions                                        495 non-null int64
Lifetime Engaged Users                                                 495 non-nul

In [30]:
df.dropna().shape

(495, 19)

In [31]:
df = df.dropna()

In [32]:
df.head(3)

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80


## Initialization

## Normalize the Input Data

Let's look at our input data. We'll use the 7 first columns as our predictors. We'll do the following two things:
- Normalize the continuous variables --> you can do this using `np.mean()` and `np.std()`
- Make dummy variables of the categorical variables (you can do this by using `pd.get_dummies`)

We only count "Category" and "Type" as categorical variables. Note that you can argue that "Post month", "Post Weekday" and "Post Hour" can also be considered categories, but we'll just treat them as being continuous for now.

You'll then use these to define X and Y. 

To summarize, X will be:
* Page total likes
* Post Month
* Post Weekday
* Post Hour
* Paid
along with dummy variables for:
* Type
* Category


Be sure to normalize your features by subtracting the mean and dividing by the standard deviation.  

Finally, y will simply be the "like" column.

In [35]:
for col in ['Post Month', 'Post Weekday', 'Post Hour']:
    vals = df[col]
    mu = vals.mean()
    stdv = vals.std()
    df[f'{col}_norm'] = (vals - mu) / stdv

In [62]:
dummies = pd.get_dummies(df[['Type', 'Category']].astype(str))

In [63]:
cols = [col for col in df if col.endswith('norm')]
cols.append('Paid')

In [64]:
#Your code here; define X and y.
X = pd.concat([df[cols], dummies], axis=1)
Y = df['Page total likes']

In [65]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 498
Data columns (total 11 columns):
Post Month_norm      495 non-null float64
Post Weekday_norm    495 non-null float64
Post Hour_norm       495 non-null float64
Paid                 495 non-null float64
Type_Link            495 non-null uint8
Type_Photo           495 non-null uint8
Type_Status          495 non-null uint8
Type_Video           495 non-null uint8
Category_1           495 non-null uint8
Category_2           495 non-null uint8
Category_3           495 non-null uint8
dtypes: float64(4), uint8(7)
memory usage: 22.7 KB


Our data is fairly small. Let's just split the data up in a training set and a validation set!  The next three code blocks are all provided for you; have a quick review but not need to make edits!

In [115]:
#Code provided; defining training and validation sets
data_clean = pd.concat([X, Y], axis=1)
np.random.seed(123)
train, validation = train_test_split(data_clean, test_size=0.2)

In [116]:
train.shape

(396, 12)

In [117]:
validation.shape

(99, 12)

In [118]:
X_val = validation.iloc[:,0:11]
Y_val = validation.iloc[:,11]    # Had to change this to eleven
X_train = train.iloc[:,0:11]
Y_train = train.iloc[:,11]

In [2]:
#Code provided; building an initial model
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=11, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=0)

In [120]:
#Code provided; previewing the loss through successive epochs
hist.history['loss'][:10]

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

Did you see what happend? all the values for training and validation loss are "nan". There could be several reasons for that, but as we already mentioned there is likely a vanishing or exploding gradient problem. recall that we normalized out inputs. But how about the outputs? Let's have a look.

In [121]:
Y_train.head()

208    132201
290    125612
286    126141
0      139441
401    107907
Name: Page total likes, dtype: int64

Yes, indeed. We didn't normalize them and we should, as they take pretty high values. Let
s rerun the model but make sure that the output is normalized as well!

## Normalizing the output

Normalize Y as you did X by subtracting the mean and dividing by the standard deviation. Then, resplit the data into training and validation sets as we demonstrated above, and retrain a new model using your normalized X and Y data.

In [122]:
#Your code here: redefine Y after normalizing the data.
Y_norm = (Y - Y.mean()) / Y.std()
Y_norm.describe().round(2)

count    495.00
mean       0.00
std        1.00
min       -2.58
25%       -0.67
50%        0.40
75%        0.82
max        1.00
Name: Page total likes, dtype: float64

In [130]:
#Your code here; create training and validation sets as before. Use random seed 123.
data_clean = pd.concat([df[cols], dummies, Y_norm], axis=1)
np.random.seed(123)
train, validation = train_test_split(data_clean, test_size = 0.2)

In [131]:
X_train = train.iloc[:,:11]
Y_train = train.iloc[:,11]
X_val = validation.iloc[:,:11]
Y_val = validation.iloc[:,11]

In [132]:
#Your code here; rebuild a simple model using a relu layer followed by a linear layer. 
    #(See our code snippet above!)
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=11, activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.compile(optimizer='sgd', loss='mse', metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32,
                 epochs=100, validation_data=(X_val, Y_val), verbose=0)

Finally, let's recheck our loss function. Not only should it be populated with numerical data as opposed to null values, but we also should expect to see the loss function decreasing with successive epochs, demonstrating optimization!

In [133]:
hist.history['loss'][:10]

[1.0714781844254695,
 0.6839352075499717,
 0.5111990739600827,
 0.4041669498188327,
 0.32762918327793933,
 0.27516485344279895,
 0.23922340601983696,
 0.21124812087627373,
 0.1911111519944788,
 0.17573078413202306]

Great! We have a converged model. With that, let's investigate how well the model performed with our good old friend, mean squarred error.

In [134]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)  

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

print("MSE_train:", MSE_train)
print("MSE_val:", MSE_val)

MSE_train: 0.01375262701952652
MSE_val: 0.01694610412951739


## Using Weight Initializers

##  He Initialization

Let's try and use a weight initializer. In the lecture, we've seen the He normalizer, which initializes the weight vector to have an average 0 and a variance of 2/n, with $n$ the number of features feeding into a layer.

In [136]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=11, kernel_initializer= "he_normal",
                activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val),verbose=0)

In [137]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [138]:
print(MSE_train)
print(MSE_val)

0.018690742963174974
0.02756687787960409


The initializer does not really help us to decrease the MSE. We know that initializers can be particularly helpful in deeper networks, and our network isn't very deep. What if we use the `Lecun` initializer with a `tanh` activation?

## Lecun Initialization

In [139]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=11, 
                kernel_initializer= "lecun_normal", activation='tanh'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=0)

In [140]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [141]:
print(MSE_train)
print(MSE_val)

0.019103912882146384
0.020530996623646193


Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impace of various optimization algorithms.

## RMSprop

In [142]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=11, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "rmsprop" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [143]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [144]:
print(MSE_train)
print(MSE_val)

0.008728219844128762
0.013023289748297093


## Adam

In [145]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=11, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "Adam" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [146]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [147]:
print(MSE_train)
print(MSE_val)

0.01283465794572557
0.018508882191963224


## Learning Rate Decay with Momentum


In [148]:
np.random.seed(123)
sgd = optimizers.SGD(lr=0.03, decay=0.0001, momentum=0.9)
model = Sequential()
model.add(layers.Dense(8, input_dim=11, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= sgd ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [149]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [150]:
print(MSE_train)
print(MSE_val)

0.009343972371333173
0.012344414868889276


## Additional Resources
* https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/Consumer_complaints.ipynb  

* https://catalog.data.gov/dataset/consumer-complaint-database  

* https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/  

* https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/  

* https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/  

* https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network  

* https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/


## Summary  

In this lab, we began to practice some of the concepts regarding normalization and optimization for neural networks. In the final lab for this section, you'll independently practice these concepts on your own in order to tune a model to predict individuals payments to loans.